# Spatio-Temporal Analysis with grass.jupyter

As part of a GRASS mini grant, we've been adding visualization functions for time space datasets (strds and stvds). You can find out more about the project and follow the progress on the [GRASS wiki page](https://trac.osgeo.org/grass/wiki/GSoC/2021/JupyterAndGRASS/MiniGrant2022).

This interactive notebook is available online thanks to the [https://mybinder.org](Binder) service. To run the select part (called a *cell*), hit `Shift + Enter`.

## Download Dataset

In [ ]:
import subprocess
import sys
import shutil

# Download zip
!curl https://grass.osgeo.org/sampledata/north_carolina/nc_climate_spm_2000_2012.zip -o ../../../data/nc_climate_spm_2000_2012.zip

# Unpack zip to grassdata
shutil.unpack_archive(
    "../../../data/nc_climate_spm_2000_2012.zip",
    "../../../data/nc_basic_spm_grass7/",
    "zip",
)

## Start GRASS GIS

In [ ]:
# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/data/nc_basic_spm_grass7/climate_2000_2012")

Create empty space time datasets

In [ ]:
gs.run_command(
    "t.create",
    output="tempmean",
    type="strds",
    temporaltype="absolute",
    title="Average temperature",
    description="Monthly temperature average in NC [deg C]",
)

gs.run_command(
    "t.create",
    output="precip_sum",
    type="strds",
    temporaltype="absolute",
    title="Preciptation",
    description="Monthly precipitation sums in NC [mm]",
)

Create list of rasters to be registered to empty space time datasets

In [ ]:
tempmean_list = gs.read_command(
    "g.list", type="raster", pattern="*tempmean", separator="comma"
).strip()

precip_list = gs.read_command(
    "g.list", type="raster", pattern="*precip", separator="comma"
).strip()

Register the rasters to the space time dataset created above

In [ ]:
gs.run_command(
    "t.register",
    input="tempmean",
    type="raster",
    start="2000-01-01",
    increment="1 months",
    maps=tempmean_list,
    flags="i",
)

gs.run_command(
    "t.register",
    input="precip_sum",
    type="raster",
    start="2000-01-01",
    increment="1 months",
    maps=precip_list,
    flags="i",
)

Extract a small subset for visualization

In [ ]:
gs.run_command(
    "t.rast.extract",
    input="precip_sum",
    output="precip_sum_2010",
    where="start_time >= '2010-01-01' and start_time < '2011-01-01'",
)

Set the color table for all rasters in the space time raster dataset

In [ ]:
gs.run_command("t.rast.colors", input="precip_sum_2010", color="precipitation_monthly")

## Temporal Visualizations

The `TimeSeriesMap` class contains visualization functions for GRASS space time dataset (strds or stvds) with either a time slider using `show` or by exporting as a GIF with `save`.

In [ ]:
precip_map = gj.TimeSeriesMap(use_region=True)
precip_map.add_raster_series("precip_sum_2010")
precip_map.d_legend(color="black", at=(10, 40, 2, 6))  # Add legend
precip_map.d_vect(map="boundary_state", fill_color="none")
precip_map.d_barscale()
precip_map.show()

We can also display the space time dataset as a GIF with IPython and `save`.

In [ ]:
from IPython.display import Image

Image(
    precip_map.save(
        "image.gif", duration=500, label=True, text_size=16, text_color="gray"
    )
)

Here is the same example but with two of the rasters unregistered, creating a dataset with variable timesteps.

In [ ]:
gs.run_command(
    "t.unregister",
    type="raster",
    input="precip_sum_2010",
    maps="2010_02_precip,2010_08_precip",
)
print(
    gs.read_command("t.rast.list", input="precip_sum_2010", columns="name,start_time")
)

In [ ]:
gaps_map = gj.TimeSeriesMap()
gaps_map.add_raster_series("precip_sum_2010", fill_gaps=False)
gaps_map.d_legend(color="gray", at=(10, 0, 30, 0))  # Add legend
gaps_map.show()  # Create TimeSlider

By default, `fill_gaps=False`, so there is are blank images where we removed rasters. By setting `fill_gaps=True`, we will see the gap filled by the previous time step.

In [ ]:
filled_map = gj.TimeSeriesMap()
filled_map.add_raster_series("precip_sum_2010", fill_gaps=True)
filled_map.d_legend(color="gray", at=(10, 0, 30, 0))  # Add legend
filled_map.show()  # Create TimeSlider